# Practical Advanced AI Project

In this notebook, we’ll build a **complete advanced AI pipeline** : from data processing and model training to explainability and deployment. 

This capstone project combines concepts from previous modules like:
- Transfer Learning
- Explainable AI (XAI)
- Model Compression
- Deployment and Monitoring

Let’s demonstrate this using an **image classification problem** : classifying images from the CIFAR-10 dataset using a fine tuned CNN model.

## 🎯 Project Workflow

1. Load and preprocess CIFAR-10 data
2. Fine-tune a pre-trained model (ResNet18)
3. Evaluate the model using advanced metrics
4. Apply explainability tools (LIME / Grad-CAM)
5. Compress and optimize the model
6. Deploy using FastAPI (conceptual overview)
7. Conclude with key learnings

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

## 🧹 1. Data Loading and Preprocessing

In [2]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=32, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=32, shuffle=False)

classes = trainset.classes
print('Classes:', classes)

## 🧠 2. Fine-tuning a Pre-trained Model (ResNet18)

In [3]:
model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False  # freeze base layers

# modify final layer
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 10)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

## ⚙️ 3. Model Training

In [4]:
epochs = 2  # use small number for demo
for epoch in range(epochs):
    running_loss = 0.0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(trainloader):.4f}')

## 📈 4. Evaluation and Explainability

In [5]:
from sklearn.metrics import classification_report

model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

print(classification_report(y_true, y_pred, target_names=classes))

### 🔍 Grad-CAM Visualization (Model Explainability)

In [6]:
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

target_layer = model.layer4[-1]
cam = GradCAM(model=model, target_layers=[target_layer])

images, labels = next(iter(testloader))
input_tensor = images[0:1].to(device)

grayscale_cam = cam(input_tensor=input_tensor, targets=[ClassifierOutputTarget(1)])
grayscale_cam = grayscale_cam[0, :]
rgb_img = images[0].permute(1,2,0).numpy()
visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)

plt.imshow(visualization)
plt.title('Grad-CAM Explanation')
plt.axis('off')
plt.show()

## 🪶 5. Model Compression and Export

In [7]:
compressed_model_path = 'compressed_resnet18.pth'
torch.save(model.state_dict(), compressed_model_path)
print('✅ Model saved and ready for deployment!')

## 🌐 6. Deployment with FastAPI (Concept Overview)

Example API structure:

```python
from fastapi import FastAPI, UploadFile
from PIL import Image
import torch

app = FastAPI()
model = torch.load('compressed_resnet18.pth')

@app.post('/predict')
async def predict(file: UploadFile):
    img = Image.open(file.file)
    # preprocess and predict
    return {"prediction": "cat"}
```

Deploy via Docker or cloud services like AWS, GCP, or Azure ML.

## 🧭 7. Key Learnings

- Combined advanced concepts: transfer learning, explainability, and deployment.
- Used **Grad-CAM** for model transparency.
- Saved and compressed the model for edge use.
- Prepared an API-based deployment setup.

**This project concludes the Advanced Concepts module and ties together all core ML to MLOps skills.** 🎓